<a href="https://colab.research.google.com/github/grams3003/ML-4430/blob/main/ML_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
%cd 'drive/MyDrive/ML'

/content/drive/MyDrive/ML


In [8]:
!pip install talos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
! pip install torch_optimizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.4 MB/s eta 0:00:00


In [38]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import pickle
import talos
import matplotlib.pyplot as plt
import numpy as np 
import torch_optimizer 
from sklearn.metrics import f1_score

In [10]:

with open('train', 'rb') as file:
    train_dict = pickle.load(file, encoding='bytes')

with open('test', 'rb') as file:
    test_dict = pickle.load(file, encoding='bytes')

X_train = torch.tensor(train_dict[b'data'])
y_train = torch.tensor(train_dict[b'coarse_labels'])

X_test = torch.tensor(test_dict[b'data'])
y_test = torch.tensor(test_dict[b'coarse_labels'])

y_train = F.one_hot(y_train, 100)
y_test = F.one_hot(y_test, 100)

In [ ]:
print(X_train[0].size())
print(X_train.size())

torch.Size([3072])
torch.Size([50000, 3072])


In [28]:
p = {
    'units': [120, 240],
    'hidden_activations': ['relu', 'sigmoid'],
    'activation': ['softmax', 'sigmoid'],
    'loss': ['mse', 'categorical_crossentropy'],
    'optimizer': ['adam', 'adagrad'],
    'batch_size': [1000, 2000]
}

epochs = 200

In [12]:
# class MLP(nn.Module):
#   def __init__(self, input_dim, output_dim, p):
#     super().__init__()

#     self.fc1 = nn.Linear(input_dim, 2000)
#     self.fc2 = nn.Linear(2000, 1500)
#     self.fc3 = nn.Linear(1500, 1000)
#     self.fc4 = nn.Linear(1000, 500)
#     self.fc5 = nn.Linear(500, 250)
#     self.out = nn.Linear(250, output_dim)

#   def forward(self, x):
#     h1 = nn.relu(self.fc1(x))
#     h2 = nn.relu(self.fc2(h1))
#     h3 = nn.relu(self.fc3(h2))
#     h4 = nn.relu(self.fc4(h3))
#     h5 = nn.Softmax(self.fc5(h4))
#     y_pred = self.out(h5)



In [46]:
def hyper_param_tuning(X_train, y_train, x_val, y_val, params):

  def weights_init_uniform_rule(m):
    classname = m.__class__.__name__

    if classname.find('Linear') != -1:
      n = m.in_features
      y=1.0 / np.sqrt(n)
      m.weight.data.uniform_(-y,y)
      m.bias.data.fill_(0)
    
  class MLP(nn.Module, talos.utils.TorchHistory):
    
    def __init__(self, n_feature):

      super(MLP, self).__init__()

      self.fc1 = nn.Linear(3072, 2000)
      self.fc2 = nn.Linear(2000, 1500)
      self.fc3 = nn.Linear(1500, 1000)
      self.fc4 = nn.Linear(1000, 500)
      self.fc5 = nn.Linear(500, 250)
      self.out = nn.Linear(250, 100)

    def forward(self, x):
      h1 = F.relu(self.fc1(x))
      h2 = F.relu(self.fc2(h1))
      h3 = F.relu(self.fc3(h2))
      h4 = F.relu(self.fc4(h3))
      h5 = F.Softmax(self.fc5(h4))
      y_pred = self.out(h5)      

  net = MLP(X_train.shape[1])
  net.apply(weights_init_uniform_rule)
  loss_func = nn.CrossEntropyLoss()
  # optimizer = torch_optimizer(net, params['optimizer'])
  optimizer = torch_optimizer(net, params['optimizer'])

  net.init_history()

  for epoch in range(epochs):

    optimizer.zero_grad()

    outputs = net(X_train)

    prediction = torch.max(outputs, 1)[1]
    metric = f1_score(y_train.data, prediction.data)

    loss = loss_func(outputs, y_train)
    loss.backward()
    optimizer.step()

    output_val = net(x_val)
    prediction = torch.max(output_val, 1)[1]
    val_metric = f1_score(y_val.data, prediction.data)

    val_loss = loss_func(output_val, y_val)

    net.append_loss(loss.item())
    net.append_metric(metric)
    net.append_val_loss(val_loss.item())
    net.append_val_metric(val_metric)

  return net, net.parameters()


In [18]:
# model = MLP(3072, 100, p)

# loss = p['loss']
# optim = p['optimizer']


In [47]:
scan_object = talos.Scan(x = X_train, y = y_train, x_val = X_test, y_val = y_test, params = p, model = hyper_param_tuning, experiment_name = 'talos_output')


  0%|          | 0/64 [00:00<?, ?it/s]

TypeError: ignored